# Time Series Analysis

K. Leighly 2017

This lecture was drawn from the following sources:
 - Ivezic chapter 10
 - _Numerical Recipes_ Chapter 12 & 13
 

## Introduction

Time series data have similarities and differences compared with the data that we have looked at so far.

Some differences include:
 - The data are ordered and directional.
 - The data can show coherence, i.e., a point far distant in the future may depend in part to the state of the system at the current time.  Possibly only for white noise are data independent of one another.
 
Some similarites include:
 - We can still model with likelihoods and employ many of the methods we have already learned about.
 
In this lecture we will first talk about some very simple time series analysis methods.  Then we will move on Fourier analysis, i.e., analysis in the frequency domain, followed by a discussion of analysis in the time domain.  We will wrap up with reverberation mapping (probably next time).

## Does It Vary?

The first thing to ask when we are confronted with a light curve is: does it vary?  For example, [Leighly 1999](http://adsabs.harvard.edu/abs/1999ApJS..125..297L) present a series of ASCA light curves from a sample of Narrow-line Seyfert 1 galaxies. It is quite obvious that some vary, but in others, the errors are large and so the variability is not obvious.

One way is to fit the light curve to a constant model, and look at the $\chi^2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
temp=np.loadtxt('iras13224.dat')

print temp.shape

t=temp[:,0]
flux=temp[:,1]
err=temp[:,2]

plt.errorbar(t,flux,yerr=err,fmt='o')

In [ ]:
## Compute the variance weighted mean

wtd_mean=((flux/(err*err)).sum())/((1.0/(err*err)).sum())
print wtd_mean

## Compute the chi^2

chi2=(((flux-wtd_mean)**2)/(err*err)).sum()

print chi2

## Now compute the statistical significance of this value of chi2

from scipy import stats
dist=stats.chi2(len(flux)-1) #number of degrees of freedom is the number of points minus the number of fit parameters
print 1.0-dist.cdf(chi2) #significance


In [ ]:
temp=np.loadtxt('re1034.dat')

print temp.shape

t=temp[:,0]
flux=temp[:,1]
err=temp[:,2]

plt.errorbar(t,flux,yerr=err,fmt='o')

In [ ]:
## Compute the variance weighted mean

wtd_mean=((flux/(err*err)).sum())/((1.0/(err*err)).sum())
print wtd_mean

## Compute the chi^2

chi2=(((flux-wtd_mean)**2)/(err*err)).sum()

print chi2

## Now compute the statistical significance of this value of chi2

from scipy import stats
dist=stats.chi2(len(flux)-1) #number of degrees of freedom is the number of points minus the number of fit parameters
print 1.0-dist.cdf(chi2) #significance


## How Much Does it Vary?

Now that we have established that it varies, how can we determine how much it varies? We will get to the formal methods in a few minutes, but let's imagine that we have not very good data, i.e., there are gaps due to earth occultation, say, and the observations aren't very long for the time scale of variability (like the ASCA data).

One criterion that has been rather commonly used is the excess variance $\sigma^2_{rms}$. This metric may have first been proposed by [Nandra et al. 1997](http://adsabs.harvard.edu/abs/1997ApJ...476...70N).

$$\sigma^2_{rms} = \frac{1}{N\mu^2} \sum_{i=1}^{N}[(X_i - \mu)^2 - \sigma_i^2],$$

where $\mu$ is the mean of the light curve, and $\sigma^2_i$ is the uncertainty on each data point. Careful examination of this equation indicates that it is the variance of the data corrected for the measurement error. That is, significant measurement error is expected to produce scatter in the data that may produce variance, but doesn't reflect intrinsic variability.  Another way to say that is that the noise adds variability to the light curve, so the noise contribution is subtracted off.

Naturally, this figure of merit needs an error bar. The one that has been proposed by Nandra et al. 1997 is:

$$s_d^2 = \frac{1}{N-1} \sum_{i=1}^{N} [(X_i-\mu)^2-\sigma_i^2] - \sigma^2_{rms}\mu^2,$$

i.e., taken to be the variance in the quantity $(X_i-\mu)^2-\sigma_i^2$.

This quantity has been widely used. For example [Leighly 1999](http://adsabs.harvard.edu/abs/1999ApJS..125..297L) used it to investigate the X-ray variability of a subclass of Seyfert galaxies called Narrow-line Seyfert 1 galaxies (NLS1s). It was discovered that for a given X-ray luminosity (taken to be a measure of a the accretion rate)  NLS1s show higher excess variance than their broad-line counterparts. This was taken as evidence that NLS1s are a class of objects that have higher accretion rates than BLS1s.

![Leighly 1999a](http://iopscience.iop.org/article/10.1086/313277/fulltext/fg3.h.gif)

It must be noted that excess variance is not a bias-free estimator by any means. It has all sorts of problems. For example, quasars exhibit red noise (we will discuss this later), which means that the excess variance observed from a single quasar can vary dramatically depending on when it is observed.  More concretely, a long light curve broken into two pieces can produce supposedly significantly different excess variance values, even if the underlying stocastic process is the same. 

In addition, comparison of excess variance values requires that the sampling (i.e., gaps and binning) and length of the observations (due to the red noise issue) be roughly the same. This is because a property of red noise is that the longer you look, the greater amplitude variability you see.  So the uncertainty listed above is certainly a lower limit. For additional discussion of the problems using excess variance, please see [Allevato et al. 2013](http://adsabs.harvard.edu/abs/2013ApJ...771....9A).

## What about bursts?

Some types of objects, e.g., the black hole at the center of our galaxy, exhibit bursts of X-rays. Detection of variability in lightcurves from such objects is challenging. 
- The object may be faint (few photons) and your data may consist of only photon arrival times.  In addition, use of $\chi^2$ assumes normally-distributed errors.  You can bin the data in time, but you need to bin so that you have more than $\sim 15$ photons per bin; otherwise, the errors will be Poission distributed and $\chi^2$ can't be used. 
- But binning data can average over the interesting phenomena. 
- Alternatively, you may have good statistics, but the bursts may be rare, so if you compute something like excess variance over the whole light curve, you may still show no significant variability detection.

One of the ways that has been proposed to detect bursts is the Bayesian Blocks method proposed by [Scargle 1998](http://adsabs.harvard.edu/abs/1998ApJ...504..405S) and [Scargle et al. 2013](http://adsabs.harvard.edu/abs/2013ApJ...764..167S).  It is also discussed in Ivezic section 5.7.2.

In this algorithm, the data are segmented into blocks with the borders between two blocks set by "change points". A Bayesian analysis based on the Poission statistics within each block means that an objective function, the log-likeliness fitness function, can be defined for each block:

$$F(N_i,T_i) = N_i (\log N_i - \log T_i),$$

where $N_i$ is the number of points in block $i$, and $T_i$ is the width of block $i$ (i.e., the duration of the block). As we know, log likelihoods are additive, so the fitness function for any set of blocks is the sum of the fitness functions for each individual block.  The analysis then involves estimating the maximum likelihood.   This may not be trivial if there are lots of points.

The figure below (Ivezic 5.21) shows an example of the implementation of this formalism, compared with an alternative one (Kunth Histogram) which uses blocks of equal length. You can see how it is able to detect narrow burst-like features. Note that the Bayesian Blocks algorithm is available in the astroML package.

![Ivezic figure from chapter 5](http://www.astroml.org/_images/fig_bayes_blocks_1.png)

If you are interested in this technique, I encourage you to look at [Scargle et al. 2013](http://adsabs.harvard.edu/abs/2013ApJ...764..167S).

## Fourier Analysis

Now let's say that you have somewhat better data, and you want to do some serious time series analysis. Perhaps you are interested in looking for a periodic signal, or maybe you are interested in the slope of the power spectrum or, if you have a power-law power spectrum, whether there are any breaks that might give you a hint about time scales in the source. Fourier analysis seems to be an immediate possible choice.

Those of you who have had math methods are familiar with the Fourier transform. The Fourier transform of a function $h(t)$ is defined as (note: I am following Ivezic in this lecture; as they note, they use arguably non-standard notation, e.g., they use $2\pi f$ instead of $\omega$, etc):

$$H(f)=\int_{-\infty}^{\infty} h(t) \exp(-i 2 \pi f t) dt,$$
and the inverse transform is:

$$h(t) = \int_{-\infty}^{\infty} H(f) \exp(i 2 \pi ft) df,$$

where $f$ is frequency and $t$ is time.

More often, we have time series sampled at discrete times, and we use the _discrete Fourier transform_. For a continuous light curve $h(t)$ which is sampled at $t_j=t_0,\ldots,t_{N-1}$ yielding a light curve with amplitudes $h_0,\ldots,h_{N-1}$:

$$H_k =\sum_{j=0}^{N-1} h_j \exp[-i 2 \pi jk/N]$$

$$=\sum_{j=0}^{N-1} h_j (\cos (2 \pi jk/N)+i \sin(-2 \pi jk/N)),$$

where $k=0,\ldots,(N-1)$ is the number of points in the light curve $h_j$. The corresponding inverse discrete Fourier transform is given by

$$h_j = \frac{1}{N} \sum_{k=0}^{N-1} H_k \exp[i 2 \pi j k/N]$$

$$=\frac{1}{N}\sum_{k=0}^{N-1} H_k \left (\cos (2 \pi jk/N)+i \sin(2 \pi jk/N) \right),$$

for $j=0,\ldots,(N-1)$.

A Fourier transform essentially seeks to build up a time series in term of the sums of sines and cosines. 
- If the sum runs from minus to plus $\infty$, i.e., an infinite number of $H_k$ are produced, then the time series is modelled exactly. 
- But if the summation is cut off, then the resulting reconstition of the time series will be better or worse, depending on the nature of the time series. 
- The closer to a sinusoid the signal is, the fewer frequencies contribute significantly (i.e., have $H_k$ significantly different from zero). 
- A sharp feature in the light curve requires high frequency components.

Just to remind you of what you know about the Fourier Transform, see [this webpage](http://mri-q.com/fourier-transform-ft.html).  The Ivezic Figure 10.1 below shows the attempt to model a RR Lyrae light curve with a truncated Fourier transform.

![ivezic chapter 10 figure 1](http://www.astroml.org/_images/fig_rrlyrae_reconstruct_1.png)

- In a way, the Fourier transform is something like a "fit" of a time series in terms of frequency components (although, the Lomb-Scargle periodogram is more like a fit, as we will see below). 
- The Fourier transform is a complex number, where one component can be viewed as holding information about the _amplitude_ of a frequency component, and the other holds information about the _phase_ of a frequency component. This can be understood by the fact that $\cos (2 \pi jk/N)+i \sin(-2 \pi jk/N)$ is equivalent to $\cos(2 \pi f t - \phi(t))$, i.e., remember that the phase can be understood as the rotation of the unit vector through the complex plane. What it means is that the Fourier components making up the light curve need not be aligned in phase.

Often, we are not interested in the phase, but only interested in the amplitude (although that is not always the case; e.g., [Maccarone et al. 2011](http://adsabs.harvard.edu/abs/2011MNRAS.413.1819M) discuss analysis using the bispectrum, which explores the relations between relative phases of the Fourier spectrum at different frequencies).  

The amplitude of the Fourier transform is expressed as the power spectrum:

$$PSD(f) = \bar{H} H = | H(f)|^2 + |H(-f)|^2.$$

The power spectral density relays the amount of power (i.e., the amount of variability) at various frequencies. 

A nice paper published by [Press in 1978](http://adsabs.harvard.edu/abs/1978ComAp...7..103P) shows the difference between 
- white noise ($P(f) \propto f^0$), which has the same power at low and high frequencies
- random walk ($P(f) \propto f^{-2}$), also called red noise, which has a lot of power at low frequencies
- flicker or $1/f$ noise, which has power at both high and low frequencies. 

As Press points out, the unique property of flicker noise is that the power is divergent to both low and high frequencies. We will discuss so-called stocastic noise later.

When working with power spectra and in the frequency domain, it is useful to try to visualize how low frequencies (long wavelengths) correspond to large-scale features, while high frequencies (small wavelengths) correspond to small-scale features).  The video below shows an illustration of the deconvolution of the cosmic microwave background in terms of spherical harmonics.  You can see how the higher frequencies correspond to smaller size scales.

A good illustration of how a power spectrum works is [here](https://www.youtube.com/watch?v=D_aEbZFPC9U).

### Examples

Let's use astroml codes to generate some light curves and take their power spectra.

In [ ]:
from astroML.time_series import generate_power_law
from astroML.fourier import PSD_continuous


In [ ]:
N = 1024
dt = 0.01
factor = 100

t = dt * np.arange(N)

##  First try white noise

beta=0.0

x=factor*generate_power_law(N, dt, beta)
x2=factor*generate_power_law(N, dt, beta)

In [ ]:
pylab.rcParams['figure.figsize'] = (15, 6)


plt.subplot(1,2,1)

plt.plot(t,x)
plt.ylim(-10,10)
plt.subplot(1,2,2)

plt.plot(t,x2)
plt.ylim(-10,10)

In [ ]:
# now compute the power spectrum.  PSD_continuous sets up the 
# frequency spectrum for you.

f, PSD = PSD_continuous(t, x)
f2, PSD2 = PSD_continuous(t, x2)

In [ ]:
plt.subplot(1,2,1)

plt.loglog(f,PSD)
plt.ylim(0.001,10)

plt.subplot(1,2,2)

plt.loglog(f,PSD2)
plt.ylim(0.001,10)

print f[1]


### Modeling power spectra

The above power spectrum appears flat, and indeed it should, given that it is from a white noise process.  
But what if you wanted to ascertain that, i.e., fit the power spectrum with a model?  In  particular, a power law model would be linear in log frequency, i.e., 

$$log PSD = a + b \times log f.$$

One problem is that the amount of information available at low frequencies is much less than the amount of information available at high frequencies.  This can be understood intuitively;  the lowest frequency is $\sim 0.1$, which corresponds to a period of $10$, i.e., the full range of the light curve.  If you were to "fit" a sine or cosine wave with that period onto the light curves, only one cycle would stretch from end to end.  In contrast, a lot of information is available to constrain the phase and amplitude of e.g., $f=1.0$, i.e., $P=1.0$, as 10 cycles would stretch across the light curve.

So what to do?  [Papadakis & Lawrence 1993](http://adsabs.harvard.edu/abs/1993MNRAS.261..612P) discuss this problem.

One possibility is to break a long time series in to many shorter ones, take power spectra of each segment, and then average, taking the uncertainty as the standard deviation of points at each frequency.  But this procedure ends up losing the low frequencies.  

An alternative method is explored in Papadakis & Lawrence.  They show that you can take the base-10 logarithm of the frequency and the PSD, and then you can bin those values.  They show, through simulations, that this gives a relatively unbiased estimate of the power spectral slope through a model such as the one above, **specificaly for the case where the underlying power spectrum is pink or red (i.e., more power at low frequencies).**

Next let's try the $1/f$ noise (flicker or pink noise) and $1/f^2$ noise (red noise or random walk).

In [ ]:
beta=1.0

x=factor*generate_power_law(N, dt, beta)
x2=factor*generate_power_law(N, dt, beta)

plt.subplot(1,2,1)

plt.plot(t,x)
plt.ylim(-1.5,1.5)
plt.subplot(1,2,2)

plt.plot(t,x2)
plt.ylim(-1.5,1.5)

f, PSD = PSD_continuous(t, x)
f2, PSD2 = PSD_continuous(t, x2)

In [ ]:
logtempf=np.arange(-1.0,2.0,0.1)

tempf=10.0**logtempf
logtempy=-logtempf
tempy=10.0**logtempy

plt.subplot(1,2,1)

plt.loglog(f,PSD)
plt.loglog(tempf,tempy)
plt.ylim(0.0001,10)

plt.subplot(1,2,2)

plt.loglog(f,PSD2)
plt.ylim(0.0001,10)



In [ ]:
### Note that the zeroth element is ignored.

logf=np.log10(f[1:])
logPSD=np.log10(PSD[1:])
logPSD2=np.log10(PSD2[1:])
plt.plot(logf,logPSD)
plt.plot(logf,logPSD2)

In [ ]:
print logPSD.shape

## Write a function that will bin the log of the PSD as a funciton of log frequency, and return the 
## values and uncertainties, where uncertainty is the standard deviation in a bin.

def binPSD(logf,logPSD,NumPerBin):
    numout=logPSD.shape[0]/NumPerBin
    logfbin=np.zeros(numout)
    logPSDbin=np.zeros(numout)
    logPSDbinSigma=np.zeros(numout)
    for i in arange(numout):
        logfbin[i]=logf[i*NumPerBin:(i+1)*NumPerBin].mean()
        logPSDbin[i]=logPSD[i*NumPerBin:(i+1)*NumPerBin].mean()
        logPSDbinSigma[i]=logPSD[i*NumPerBin:(i+1)*NumPerBin].std()
    return logfbin,logPSDbin,logPSDbinSigma

In [ ]:
NumPerBin=15

logfbin,logPSDbin,logPSDbinSigma=binPSD(logf,logPSD,NumPerBin)
logfbin,logPSDbin2,logPSDbinSigma2=binPSD(logf,logPSD2,NumPerBin)


plt.errorbar(logfbin,logPSDbin,yerr=logPSDbinSigma)
plt.errorbar(logfbin,logPSDbin2,yerr=logPSDbinSigma2)

In [ ]:
from scipy.optimize import curve_fit

def line_model(x,const,slope):
    return const+slope*x

result,cov=curve_fit(line_model,logfbin,logPSDbin,sigma=logPSDbinSigma)
result2,cov2=curve_fit(line_model,logfbin,logPSDbin2,sigma=logPSDbinSigma2)

In [ ]:
print result,cov
print result2,cov2

In [ ]:
beta=2.0

x=factor*generate_power_law(N, dt, beta)
x2=factor*generate_power_law(N, dt, beta)

plt.subplot(1,2,1)

plt.plot(t,x)
plt.ylim(-0.75,0.75)
plt.subplot(1,2,2)

plt.plot(t,x2)
plt.ylim(-0.75,0.75)

f, PSD = PSD_continuous(t, x)
f2, PSD2 = PSD_continuous(t, x2)

In [ ]:
logtempy=-2.0*logtempf
tempy=10.0**logtempy

plt.subplot(1,2,1)

plt.loglog(f,PSD)
plt.ylim(0.0000001,10)
plt.loglog(tempf,tempy)

plt.subplot(1,2,2)

plt.loglog(f,PSD2)
plt.ylim(0.0000001,10)


In [ ]:
logf=np.log10(f[1:])
logPSD=np.log10(PSD[1:])
logPSD2=np.log10(PSD2[1:])
plt.plot(logf,logPSD)
plt.plot(logf,logPSD2)

In [ ]:
NumPerBin=15

logfbin,logPSDbin,logPSDbinSigma=binPSD(logf,logPSD,NumPerBin)
logfbin,logPSDbin2,logPSDbinSigma2=binPSD(logf,logPSD2,NumPerBin)


plt.errorbar(logfbin,logPSDbin,yerr=logPSDbinSigma)
plt.errorbar(logfbin,logPSDbin2,yerr=logPSDbinSigma2)

In [ ]:

result,cov=curve_fit(line_model,logfbin,logPSDbin,sigma=logPSDbinSigma)
result2,cov2=curve_fit(line_model,logfbin,logPSDbin2,sigma=logPSDbinSigma2)

print result
print result2

## Important facts about Power Spectra and Fourier Transforms

### Parseval's theorem:

The total power is the same whether computed in the time domain or the frequency domain:

$$P = \int_0^{\infty} PSD(f) df = \int_{-\infty}^{\infty} |h(t)|^2
dt.$$

In effect, this means that one can obtain the _same information in the frequency domain as in the time domain_. Moreover, any analysis that can be carried out in the frequency domain can be carried out in the time domain. So, for example, in the frequency domain, we have a computational entity called a _periodiogram_, and the time domain, its counterpart is called the _structure function_.  We will discuss analysis in the time domain later.

### Convolution Theorem:

The convolution of two functions is given by

$$(a \star b) (t) = \int_{-\infty}^{\infty} a(t^\prime)
b(t-t^\prime)dt^\prime.$$

Convolution is an important and unavoidable feature of time series analysis. 

The ideal data from the astronomical object is purely continuous (if such a thing could exist), and has no beginning or end. 

- In reality, the observation has a beginning and end, which can lead to a problem called "red noise leak" that occurs because the power spectrum assumes a periodic boundary conditions. (See [Papadakis & Lawrence 1993](http://adsabs.harvard.edu/abs/1993MNRAS.261..612P) for a discussion.) 
- In addition, observations can be interrupted because of earth occultations or poor weather. 

The resulting window function is then convolved with the light curve of the object.

The convolution theorem states that if $h = a \star b$, then the Fourier transforms of $h$, $a$, and $b$ are related by:

$$H(f)=A(f) B(f).$$

This is a very handy result outside of time series analysis. So, for example, say I have a theoretical spectrum and I want to smear it with a Gaussian. A handy way to do that is to take the fourier transform of the theoretical spectrum, multiply by a Gaussian (since the fourier transform of a gaussian is a gaussian) and then take the inverse fourier transform.

Figure 10.2 from Ivezic below shows the effect of a square window on a light curve.

![Ivezic 10.2](http://www.astroml.org/_images/fig_convolution_diagram_1.png)

         Figure 10.2 from Ivezic. This figure should be read clockwise from the top left panel. The signal (solid line) basically smeared with the top-hat window function (shaded areas). The fourier transform of the window and the light curve are shown in the top right panel, the product of the two in the lower right, and the inverse fourier transform in the lower left shows a very smoothed function.

If the window function is known, can one deconvolve its effect by dividing the power spectrum of the observed light curve by the window function's power spectrum to extract the intrinsic power spectrum? It is possible, but it can only be done exactly if there is no noise.  So this type of analysis should be undertaken with caution (and plenty of simulations).

## The Nyquist Sampling theorem and Aliasing

Nyquist sampling and aliasing have to do with the fact that if you sample or bin data with a certain cadence, you are not going to be able to say anything about what is happening at a higher cadence. (Note that the same effect happens with images - an image pixelated with a certain size pixel no longer has information about what is happening at size scales smaller than that pixel.) 

To relate the effect back to our idea of the Fourier transform being a kind of "fitting" of a light curve to sines and cosines with different amplitudes and phases, consider fitting the light curve to a frequency component with period much smaller than the time sampling bin size: the amplitude cannot be constrained. Indeed, for evenly sampled data, with bin size of $\Delta t$, the smallest frequency that you can constrain is $1/2 \Delta t$. This frequency is called the Nyquist frequency.

The existence of the Nyquist frequency leads to a phenomenon called _aliasing_. Imagine that you have a light curve in which the sampling time is $\Delta t$. First, imagine that you have the enviable situation where there is no variability on time scales shorter than $\Delta t$. That is the same as saying that there is no power in the signal for frequencies shorter than the Nyquist frequency. In this case, all the information in the light curve is contained in the Fourier transform, and the light curve can be reconstituted exactly from the inverse Fourier transform (if there is no noise).

Now imagine that your lightcurve is undersampled, and there is significant variability on time scale shorter than your sampling time. 

- One might think that this should not be a problem for the Fourier transform in the range where it can be constrained. However, that is not the case. _Power from the shorter time scales is aliased into the observed frequency range, muddying the observed power spectrum_. 

- One can understand this by thinking about several adjacent points in the light curve. If there is no shorter frequency power, their amplitude is exactly what is expected from the underlying intrinsic power spectrum. 

- But if there is shorter frequency power, those points will be moved up or down by that power. That causes an amplification of power over the intrinsic, especially at high frequencies for steep power spectra.

Figure 10.3 from Ivezic, reproduced below, illustrates the idea of aliasing. Figure 12.1.1 in Numerical Recipes illustrates aliasing well.

![Ivezic Figure 10.3 top](http://www.astroml.org/_images/fig_FFT_aliasing_1.png)

     Figure 10.3 from Ivezic. This panel shows a well-sampled light curve. The frequency of sampling is greater than the frequency of the variability. Top left: a guassian and sampling window. Top right: the fourier transform of the signal and the window. Bottom left: the sampled data. Bottom right: convolution of the window and the signal.

![Ivezic Figure 10.3 bottom](http://www.astroml.org/_images/fig_FFT_aliasing_2.png)

     Figure 10.3 from Ivezic. A poorly sampled signal.

Regardless, we can compute the Power Spectral Density (PSD) and it can be useful, especially if we supplement the analysis of the observations with simulated light curves having known noise and power spectral shape. We can write the equation for the PSD for any discrete and evenly sampled function:

$$PSD(f_k) = 2 (\frac{T}{N})^2 \left [ \left (\sum_{j=0}^{N-1} h_j \cos(2 \pi f_k
t_j) \right )^2 + \left (\sum_{j=0}^{N-1} h_j \sin(2 \pi f_k t_j)\right)^2\right ],$$

where $f_k=k/(N\Delta t)$ for $k \leq N/2$ and $f_k = (k-N)/(N\Delta t)$ for $k \ge N/2$, and $N$ is the number of points in the light curve.

### Noise

All data has some level of uncertainty. The effect of the uncertainty is to shift the light curve amplitudes randomply higher or lower. 

There are many different kinds of noise possible, but a common one is simply measurement noise induced by the fact that the data has a non-infinite signal-to-noise ratio. An example would be Poission noise, i.e., the fact that the uncertainty in an observation of $N$ photons is $\sqrt{N}$. Such measurement noise is uncorrelated; the amount of noise contributed to point $\{i\}$ is independent of the amount of noise contributed to point $\{i+1\}$. _Such noise is white, and its effect is generally to flatten the high frequency end of the power spectrum_ (if the power spectrum is steep).



### The Window Function

The effect of the window function, as mentioned, is to convolve the signal from the object with the window, and that is equivalent to multiplying the Fourier transform of the light curve by the Fourier transform of the window function (via the convolution theorem).

The window function can have a profound effect on the power spectral density. For example, if the light curve is obtained by an X-ray satellite in low-earth orbit, it will typically have gaps due to earth occultation every $\sim 96$ minutes. That will induce a strong periodic peak in the power spectrum. See [Done et al. 1992](http://adsabs.harvard.edu/abs/1992ApJ...400..138D) for an illustration of analysis of an X-ray light curve using a power spectrum.

Figure 10.4 shows the power spectrum of some unevenly sampled but periodic data, and the power spectrum of the window function. Here, they compute the power spectrum of the evenly-sampled data and convolved it with the window function (examine the code associated with the figure).

![Ivezic Figure 10.4](http://www.astroml.org/_images/fig_FFT_sampling_1.png)

     Top left: the intrinsic light curve, with the randomly sampled data. Bottom left: the window function generated by the randomly sampled data; basically, the window function is set to 1 at the points where the data is kept. Bottom right: the power spectrum from the window light curve, an evenly sampled function with 1s where the data were kept, and zero elsewhere. Top right: the product of the power spectrum of the periodic signal, multiplied by the power spectrum of the window.

### Harmonics

It is rare for periodic signals to be perfect sine waves of one frequency (the exception may be binaries or orbiting planets).  Generally, there is some pulse profile that is not a perfect sinusoid (For example, the RR Lyrae light curve in Ivezic Fig 10.1). _The effect of the non-sinusoidal behavior will be to transfer some of the signal to harmonics of the primary frequency_. This will reduce the amplitude of the primary frequency, and depending on how strong the periodic signal is compared with the noise, the signal may become undetectable. In this case, period-folding and other time-domain techniques may be advisable.

### Fast Fourier Transform

The fast fourier transform (FFT) is an algorithm for computing discrete Fourier transform. Naturally, the Fourier transform can be computed by summing up sines and cosines, as above, but it can be a time-consuming process, requiring $O(N^2)$ computations (i.e., for each of $N$ frequencies, $N$ data points must be summed). The FFT is faster, requring $O(N \log N)$ operations. It is very commonly used. The algorithm is discussed in Numerical Recipes, and the there is an implementation in scipy called fftpack. It is noted that it is most efficient when the number of points in the light curve are a power of 2. Typically, if that is not true, the light curve is padded with zeros to make it be true.

An example of an application of the FFT is shown in Figure 10.5 from Ivezic, reproduced below. Note however that it doesn't show the use of the FFT; that is bundled into astroML.fourier.PSD_continuous which makes the frequency array and computes the power spectrum.  Examine the code associated with the figure to see this.

![Ivezic 10.5](http://www.astroml.org/_images/fig_fft_example_1.png)

    Ivezic Figure 10.5. Simulated periodic time series with different noise levels are plotted in the top panel. The lower panel shows the power spectral density computed using astroML.fourier.PSD_continuous.


## Applications - Periodic Functions

Now let's apply what we've learned to analysis of some data. We will start with a rather simpler case - analysis of periodic data.

We first note that when we talk about periodic data, we don't mean necessarily sinusoidal data. Many periodically variable phenomena, for example, vriable stars such as Cepheids, are periodic, but their light curves are far from sinusoidal. So to characterize a periodic signal, we use $y(t+P)=y(t)$, i.e., the amplitude of the light curve repeats with the period $P$. Often, periodic signals are plotted as a function of the phase $\phi$ of the period.

We will follow closely the development given in Ivezic section 10.3 for a single sinusoidal model because it uses a Bayesian framework to develop the _Lomb-Scargle periodogram_, a commonly-used tool in periodic time series analysis.

Consider time series data $(t_1,y_1),\ldots,(t_N,y_N)$. We will test whether it is periodic and estimate the period. We will assume that it is described by a single-frequency sine function with angular frequency $\omega=2\pi f = 2\pi / P$, i.e.,

$$y(t)= A \sin(\omega t +\phi) +\epsilon,$$

where $\epsilon$ is the presumably uncorrelated measurement noise.  Trigonometric identities permit us to rewrite this as:

$$y(t)=a \sin(\omega t) + b \cos(\omega t)+ \epsilon,$$

where $A=(a^2+b^2)^{1/2}$ and $\phi = \tan^{-1}(b/a)$. **The goal will be to determine $a$, $b$, and $\omega$.**

The model is fit to the data $(t_j,y_j), j=1,\ldots,N$ with the noise $\epsilon$, for the present, described by homoscedastic errors parameterized by $\sigma$.  (Another way to think of this is that if there is no periodic component, and the data has a normal distribution in $y$, then $\sigma$ would be the standard deviation of the $y$ values).  
- Note that $t_j$ are not required to be evenly sampled. 
- Note also that it is often common practice to subtract the average of the data $\bar{y}$ from the $y_j$ to avoid a large peak of power at $f=0$ (equivalent to a constant or DC offset).  
- Sometimes the data is also divided by $\sigma$ to normalize the variance to 1.

This analysis reflects one by Bretthorst, who has apparently revolutionized spectral analysis, especially with application to nuclear magnetic resonance. His book on the subject can be found [here](http://bayes.wustl.edu/glb/book.pdf).

Assuming a Gaussian error distribution, we can write the data likelihood (i.e., the data given the model parameters) as usual:

$$L=p(\{t,y\}|\omega, a, b, \sigma)$$

$$=\prod_{j=1}^{N} \frac{1}{\sqrt{2\pi}\sigma} \exp \left (-\frac{[y_j-a
\sin(\omega t_j) - b \cos(\omega t_j)]^2}{2\sigma^2} \right ).$$

We will assume uniform priors on $a$, $b$, $\omega$, and $\sigma$, and also $\omega \ge 0$, $\sigma \ge 0$. The posterior PDF then becomes:

$$p(\omega, a, b, \sigma |\{t,y\}) \propto \sigma^{-N} \exp \left (-\frac{N Q}{2\sigma^2} \right )$$

where

$$Q=V-\frac{2}{N} \left [a I(\omega)+b R(\omega) - a b M(\omega) -
\frac{1}{2} a^2 S(\omega) - \frac{1}{2} b^2 C(\omega)\right ], $$

where the following terms all depend only on $\omega$ (i.e., not on $a, b, \sigma$):

$$V=\frac{1}{N}\sum_{j=1}^{N} y_j^2,$$
$$I(\omega) = \sum_{j=1}^{N} y_j \sin(\omega t_j),$$
$$R(\omega)= \sum_{j=1}^{N} y_j \cos(\omega t_j),$$
$$M(\omega)= \sum_{j=1}^{N} \sin(\omega t_j) \cos(\omega t_j),$$
$$S(\omega)= \sum_{j=1}^{N} \sin^2(\omega t_j),$$
$$C(\omega) = \sum_{j=1}^{N} \cos^2(\omega t_j).$$

The equation for $Q$ can be simplifed under certain conditions. 

If $N>> 1$ and $\omega t_N$ is not much less than 1 (i.e., several periods must be represented in the observing interval), then $S(\omega) \approx C(\omega) \approx N/2$, and $M(\omega) << N/2$ and $Q$ becomes:

$$Q\approx V-\frac{2}{N}[a I(\omega) + b R(\omega)] + \frac{1}{2}(a^2+b^2).$$

In many cases, one wants only to identify the periodicity and the amplitude factors are not of interest (although if one wants to estimate the magnitude of the periodicity, one needs to estimate these). In that case, they can be marginalized over to produce:

$$p(\omega, \sigma |\{t,y\}) \propto \int p(\omega,a,b,\sigma|\{t,y\})
da\, db,$$
where the integration limits on $a$ and $b$ are set large enough that the exponential dominates at the edges. Then:

$$p(\omega,\sigma|\{t,y\}) \propto \sigma^{-(N-2)}
\exp \left (-\frac{NV}{2\sigma^2} + \frac{P(\omega)}{\sigma^2} \right ),$$

where the periodogram $P(\omega)$ is given by (note the resemblance to the power spectrum):

$$P(\omega) = \frac{1}{N}\left [I^2(\omega) + R^2(\omega)\right].$$

If the noise level $\sigma$ is known, then

$$p(\omega |\{t,y\},\sigma) \propto \exp \left (\frac{P(\omega)}{\sigma^2}\right ).$$

If the noise level $\sigma$ is not known, then $p(\omega,\sigma|\{t,y\})$ can be marginalized over to produce

$$p(\omega |\{t,y\}) \propto \left [1-\frac{2 P(\omega)}{N V}\right ]^{1-N/2}.$$

If we want to estimate the amplitudes, we can just take the derivative of the posterior probability distribution with respect to the ampitudes, set that equal to zero, and solve, i.e.,

$$\frac{d p(\omega, a,b,\sigma|\{t,y\})}{da} |_{a=a_0} = 0, $$

$$\frac{d p(\omega, a,b,\sigma|\{t,y\})}{db} |_{b=b_0} = 0. $$

These yield:

$$a_0=\frac{2 I(\omega)}{N}, \mbox{ } b_0=\frac{2 R(\omega)}{N}.$$

As usual, the width can be obtained using the second derivative, which for known $\sigma$ yields:

$$\sigma_a = \sigma_b = \sigma\sqrt{2/N}.$$

Returning to the determination of the best estimate of the period $\omega$. We would also like to estimate the significance of the period estimate. This is an important feature of the periodiogram, but it is important to note that it doesn't work for underlying stocastic noise.

For that, we return to $\chi^2$, recalling its relationship with the data likelihood, i.e.,

$$L=\log_e[prob(X|D,I)] = \mbox{constant} - \frac{\chi^2}{2}.$$

So, for this case,

$$\chi^2(\omega) = \frac{1}{\sigma^2} \sum_{j=1}^{N} \left [y_j -
y(t_j)\right ]^2$$
$$=\frac{1}{\sigma^2} \sum_{j=1}^{N} \left [y_j-a_0\sin (\omega t_j)-b_0
\cos(\omega t_j)\right ]^2, $$

which reduces to

$$\chi^2(\omega) = \chi^2_0\, \left [1-\frac{2}{NV} P(\omega)\right ],$$

where $\chi_0^2$ corresponds to a constant model, i.e., $y(t)=$constant, i.e., no periodic component, only noise, and is given by (remembering that the light curve was renormalized to zero by subtracting the mean):

$$\chi^2_0 = \frac{1}{\sigma^2} \sum_{j=1}^{N} y_j^2 = \frac{N
V}{\sigma^2}.$$

The periodogram can be conveniently renormalized to

$$P_{LS}(\omega) = \frac{2}{NV} P(\omega),$$

where $LS$ stands for Lomb-Scargle, i.e., the Lomb-Scargle Periodogram. With this normalization, $0 \leq P_{LS}(\omega) \leq 1$. So, the reduction in $\chi^2$, relative to the constant pure-noise model, that was provided by the inclusion of a single sinusoidal component is

$$\frac{\chi^2(\omega)}{\chi^2_0} = 1-P_{LS}(\omega).$$

Let us pause for a few minutes to contemplate this equation.  

Remember that $\chi^2$ is equivalent to the data likelihood (i.e., it arose from the minimum least squares analysis), and remember that we can compare models using the likelihood ratio.  So this equation is more or less equivalent to a likelihood ratio where one model is a constant, and the other model includes a sinusoid.   

The point is that this result shows that the periodogram is a tool with understood statistical properties that can be properly normalized etc.  

These results were apparently shown by Jaynes in 1987 for the evenly spaced case, and generalized to non-evenly spaced data by Bretthorst in 2000 (for references, see Gregory 2001).

Let the peak maximum height be located at $\omega=\omega_0$. We expect the contribution of $\chi^2$ corresponding to the peak to be $N$ (i.e., the $N$ degrees of freedom are perfectly modeled by the single sinusoid), with standard deviation of $\sqrt{2N}$ (the latter is a property of the $\chi^2$ distribution, see section 3.3.7 in Ivezic). Then the expected height of the peak is

$$P(\omega_0) = \frac{N}{4} (a_0^2 + b_0^2),$$
with standard deviation of

$$\sigma_p(\omega_0) = \frac{\sqrt{2}}{2} \sigma^2.$$

Note:

- the expected height of the peak in the periodogram depends on the amplitude of the periodicity, but does not depend on the data uncertainty $\sigma$. 
- the uncertainty in the height does depend on $\sigma$, but not on the sample size $N$.  
- the Lomb-Scargle periodogram is normalized between 0 and 1, so that the expected height is:

$$P_{LS}(\omega_0) = 1 - \frac{\sigma^2}{V}.$$

So as noise becomes negligible, $P_{LS}$ approaches the maximum value of 1. As noise increases, the peak become smaller and eventually becomes buried in the noise of the periodogram.

![Ivezic 10.14](http://www.astroml.org/_images/fig_sampling_1.png)

    Ivezic Figure 10.14. Upper left: Synthetic data characterized by a single sinusoid, and unevenly sampled. Lower left: the periodogram of the window function. Upper right: the Lomb-Scargle periodogram of the sampled data. Note the strong peak at $f=0.16$. Lower right: the periodogram when the noise is increased by a factor of 10. For comparison, the peak from the less-noisy data is included. The peak now is much shorter, blending in with the noise.

The analysis can be extended to determine the uncertainty in the best-fitting frequency. As before, we take the second derivative of the posterior probability distribution:

$$\sigma_\omega=
|\frac{d^2P(\omega)}{d\omega^2}|^{-1/2}_{\omega=\omega_0}.$$

The Gaussian approximation allows the peak to be estimated as a parabola:

$$P_{LS}(\omega)\approx
1-\frac{\sigma^2}{V}-\frac{(\omega-\omega_0)^2}{NV\sigma_\omega^2}, $$

where

$$\sigma_\omega = \omega_{1/2}[2N(V-\sigma^2)]^{-1/2},$$

is the full width half maximum of the peak.

For more details and results, see Ivezic section 10.3

Before we leave the theoretical discussion, I note that Gregory 2005 has some interesting comments on the statistical significance of the periodogram.  

Specifically, given the periodogram, $C(f_n)$, it was shown by Jaynes (1987) and generalized to non-even sampling by Bretthorst (2000), that the probability for the frequency of a periodic sinusoidal signal is given approximately by

$$p(f_n|D,I) \propto \exp\left( \frac{C(f_n)}{\sigma^2}\right )$$

where the $\sigma^2$ is  noise variance - this is taken here to be a known quantity.  This is the same thing that we showed above.  The point that Gregory 2005 makes is that exponential sharpens the peak and suppresses the noise.  One might think that this is a rather bogus change of variables, but it apparently is based on a solid statistical footing.

Gregory 2005 also goes on to state that in the case that the data uncertainty is unknown, the noise variance can be treated as a nuisance parameter and marginalized over.  Then,

$$p(f_n|D,I) \propto \left [-\frac{2 C(f_n)}{N\bar{d^2}} \right ]^{\frac{2-N}{2}},$$

where $N$ is the number of data points, and $\bar{d^2} = (1/N) \sum d_i^2$ is the mean square average of the data values.  It should be noted that the analysis assumes that an DC component has been subtracted off.

The bottom line is that the periodogram is more than just something you can compute - it has intrinsic value as a statistical parameter.

## Practical Applications of the Lomb-Scargle Periodogram

The previous section motivated the use of the Lomb-Scargle Periodogram, as it showed that it arises naturallly from a Bayesian treatment of the problem. It is, as noted before, a popular choice for time series analysis.

Here we write the form with heteroscedastic errors:

$$P_{LS}(\omega) = \frac{1}{V} \left [\frac{R^2(\omega)}{C(\omega)} + \frac{I^2(\omega)}{S(\omega)}\right ],$$

where

$$\bar{y} = \sum_{j=1}^{N} w_j y_y,$$
$$V=\sum_{j=1}^{N} w_j (y_i - \bar{y})^2,$$

with weights $w_j$

$$w_j=\frac{1}{W}\frac{1}{\sigma_j} \mbox{,   }
W=\sum_{j=1}^N\frac{1}{\sigma_j}.$$

The $R$, $I$, $C$, and $S$ are defined slightly differently:

$$R(\omega) = \sum_{j=1}^{N} w_j(y_j-\bar{y})
\cos[\omega(t_j-\tau)],$$
$$I(\omega) = \sum_{j=1}^{N} w_j(y_j-\bar{y})
\sin[\omega(t_j-\tau)],$$
$$C(\omega) = \sum_{j=1}^N w_j \cos^2[\omega(t_j-\tau)],$$
$$S(\omega) = \sum_{j=1}^N w_j \sin^2[\omega(t_j-\tau)],$$

Here, the parameter $\tau$ makes $P_{LS}$ invariant to translations of the $t$ axis, and is defined by:

$$\tan(2\omega\tau) =\frac{\sum_{j=1}^{N} w_j \sin(2 \omega
t_j)}{\sum_{j=1}^{N} w_j \cos(2 \omega t_j)}.$$

It is convenient to also define

$$M(\omega)=\sum_{j=1}^{N}w_j \sin[\omega(t_j-\tau)]
\cos[\omega(t_j-\tau)].$$

The equation that we derived before had the approximation that $C(\omega)$ and $S(\omega)$ were equal to $1/2$. Without these approximations, the estimates of $a$ and $b$ become:

$$a(\omega)=\frac{I(\omega)C(\omega)-R(\omega)M(\omega)}{C(\omega)S(\omega)-M^2(\omega)},$$
and

$$b(\omega)=\frac{R(\omega)S(\omega)-I(\omega)M(\omega)}{C(\omega)S(\omega)-M^2(\omega)}.$$

Thus, the Lomb-Scargle periodogram corresponds to a single sinusoid model, and is directly related to $\chi^2$ fit for this model.

One of the features of the Lomb-Scargle Periodogram, since it is a fit of data to sines and cosines, is that there is no real limit on the frequencies that you use.  This is in contrast to the power spectrum, which generally uses frequencies that are multiples of the Nyquist frequency. Ivezic gives some guidelines, and references for where these guidelines come from.

- the minimum frequency has to be $\omega_{min}=2\pi/(t_{max}-t_{min})$

- the maximum frequency can be a pseudo Nyquist frequency, $\omega_{max}=\mbox{median}(\pi/\Delta t)$, where $\Delta t$ is the time interval between data points. But because the gaps can be large and uneven, and in some regions the sampling can be quite good, it maybe better to sample to higher frequences.

- the frequency step can be taken to be $\delta \omega = \eta \omega_{min}$, with $\eta=0.1$, i.e., about 1/10th of the minimum frequency. A linear frequency range is recommended.

- It is noted that the small stepsize and generous $\omega_{max}$ can lead to a large number of computational frequencies. But the algorithm in astroML is fast, or at least, there is a fast version of it.

A problem with being able to compute the periodogram for any number of points is that you are subject to **false alarm probability**.  This is discussed in some detail in Ivezic section 4.6, but basically the idea is that if you test enough periods, there is a chance that you will get a reasonably high peak at some frequency by accident, even if there is no periodic signal in the data.  There is a bit clearer and more quantitative discussion in Numerical Recipes.  

But why take their word for it?  One can test the significance using simulations.  For example, a straightforward method for testing the significance of the peak is by using a bootstrap method. A simulated light curve is made by keeping the time of observations fixed (i.e., same sampling) and drawing $y$ values with replacement. The Lomb-Scargle periodogram is computed from the result. This is done many times (e.g., 1000) and the results can be used to determine the amplitude of the periodogram at each frequency that fewer than some threshold of simulated periodograms exceeds (e.g., 5%, 1%). If the peak in the periodogram from the data exceeds those lines, then that gives you the signicance of the detection.  Note that this will only work if the null hypothesis is that the variability has a normal distribution, or at least is uncorrelated. If the background variability is, say, $1/f$ noise, then more effort has to be made to construct the simulated light curves, but the concept is the same.

Ivezic Figure 10.15 shows an example of the Lomb-Scargle periodogram for a small sample ($N=30$) and errors $\sigma~0.8 A$, where $A$ is the amplitude of the single sinusoid.

![Ivezic 10.15](http://www.astroml.org/_images/fig_LS_example_1.png)

    Top: The simulated data produced by $y(t|P) = 10+\sin(2\pi t/P)$ with $P =0.3$. The uncertainties are heteroscedastic from a Gaussian distribution with width drawn from a random distribution with $0.5 \leq \sigma \leq 1.0$. Bottom: the Lomb-Scargle periodogram. The dotted ines show 1% and 5% significance levels for the highest peak, determined using 1000 bootstrap resamplings. The peak is detected with $\sim 5$% confidence.

## Generalized Lomb-Scargle Periodogram

The astroML package (also the [gatspy package](http://www.astroml.org/gatspy/)) also includes an implementation of the generalized Lomb-Scargle periodogram for cases where the average value $\bar{y}$ is not a good estimator of the mean of $y$. It is best understood through Ivezic Figure 10.16. Note - this is not the figure in the book - there was apparently a typo in the original software, so that the significance of the peak was actually much less than shown. For additional discussion of this method, including references, see Ivezic.  My impression is that there is a fair bit more to this than presented, but I didn't follow up on it.

![Ivezic 10.16](http://www.astroml.org/_images/fig_LS_sg_comparison_3.png)

    Ivezic Figure 10.16, modified version. Top: A simulated light curve from $y(t)=10+\sin(2\pi t/P)$ with $P=0.3$, corresponding to $\omega_0\approx 21$. The lightcurve is somewhat pathological, since sampling times were chosen that significantly undersample $y < 10$, when the light curve is below the mean. Bottom: The standard (grey) and generalized Lomb-Scargle periodogram (black). The periodicity is detected by the generalized, but not by the standard.

### Truncated Fourier Series Model

The idea here is that the periodicity is not described well by a single sinusoid, but rather several Fourier components (like the RR Lyrae light curve examined above). AstroML contains a routine for this case, called multiterm_periodogram. For a discussion of this method, and illuminating examples, see Ivezic.

Another method, the Gregory-Loredo (GL) algorithm, is discussed in Gregory 2005.  It looks to me like it is an epoch-folding method (i.e,. the light curve is folded at various trial periods and a metric is computed that measures how well the folded data match that period) with solid Bayesian underpinnings).  Epoch-folding has also been discussed by Davies 1990 and Davies 1991.  The interested student is encouraged to examine Gregory & Loredo 1992. 

## Examples

Let's look at some examples of time series of periodic functions.

In [ ]:
P=0.168
N=100
np.random.seed(0)
t=0.01*np.linspace(0,99,100)
print min(t),max(t)
y=100.0+20.0*sin(2.0*np.pi*t/P)
plt.plot(t,y,'o')
plt.ylim(0,150)

In [ ]:
## add some poisson noise

yobs=np.random.poisson(y)
plt.plot(t,yobs,'o')
dyobs=np.sqrt(yobs)
plt.plot(t,dyobs,'o')

In [ ]:
## Compute the power spectrum using the method of Ivezic figure 10.4

Nfreq=N/2
dt = t[1] - t[0]
df = 1. / (N * dt)
f = df+df * np.arange(Nfreq)
ynorm=y-mean(y)
yobsnorm=yobs-mean(yobs)
print min(f),max(f)
PSD_y=abs(np.fft.fft(ynorm)[:Nfreq])**2
PSD_yobs=abs(np.fft.fft(yobsnorm)[:Nfreq])**2
plt.plot(f,PSD_y)
plt.plot(f,PSD_yobs)

In [ ]:
## Try the Lomb Scargle on the same frequencies:

from astroML.time_series import lomb_scargle, lomb_scargle_BIC, lomb_scargle_bootstrap 
omega=2.0*np.pi*f
period=1.0/f
P_LS_y=lomb_scargle(t,ynorm,1,omega,generalized=True)
P_LS_yobs=lomb_scargle(t,yobsnorm,1,omega,generalized=True)
#plt.semilogx(period,P_LS_y)
plt.plot(period,P_LS_yobs)

In [ ]:
## Get the significance via bootstrap, following Ivezic figure 10.5

D = lomb_scargle_bootstrap(t, yobsnorm, dyobs, omega, generalized=True,N_bootstraps=1000, random_state=0)
sig1, sig5 = np.percentile(D, [99, 95])
print sig1,sig5

Next, let's make the periodic signal weaker relative to the offset. Since the uncertainties are Poisson, this will decrease the signal to noise of the periodic signal. For example, the amplitude of the sine in the above example is 20, and the amplitude of the uncertainty is $\sim \sqrt{100}=10$. So let's try, e.g., amplitude of 3.


In [ ]:
y2=100.0+5.0*sin(2.0*np.pi*t/P)
dy2=np.sqrt(y2)
plt.plot(t,y2,'o')
plt.ylim(0,150)
yobs2=np.random.poisson(y2)
plt.plot(t,yobs2)
dyobs2=np.sqrt(yobs2)
plt.plot(t,dyobs2)

In [ ]:
y2norm=y2-mean(y2)
yobs2norm=yobs2-mean(yobs2)
PSD_y2=abs(np.fft.fft(y2norm)[:Nfreq])**2
PSD_yobs2=abs(np.fft.fft(yobs2norm)[:Nfreq])**2
plt.plot(f,PSD_y2)
plt.plot(f,PSD_yobs2)

In [ ]:
P_LS_y2=lomb_scargle(t,y2norm,1,omega,generalized=True)
P_LS_yobs2=lomb_scargle(t,yobs2norm,1,omega,generalized=True)
plt.plot(period,P_LS_y2)
plt.plot(period,P_LS_yobs2)

In [ ]:
D = lomb_scargle_bootstrap(t, yobs2norm, dyobs2, omega, generalized=True,N_bootstraps=1000, random_state=0)
sig1, sig5 = np.percentile(D, [99, 95])
print sig1,sig5
plt.plot(period,P_LS_yobs2)

## Analysis in the Time Domain

#### The Autocorrelation and Structure Functions

As noted above, by Parseval's theorem (more or less), one can perform the same analysis in the time domain as in the frequency domain.  Moreover, some analysis, such as cross correlation, may be preferentially performed in the time domain.

We have already introduced the concept of correlation. The normalized correlation function is given by

$$CF(\Delta t) = \frac{\text{lim}_{T\rightarrow \infty} \frac{1}{T}
\int_{(T)} f(t) g(t+\Delta t) dt}{\sigma_f \sigma_g},$$

where $\sigma_f$ and $\sigma_g$ are the standard deviations of $f(t)$ and $g(t)$. The correlation function informs about the time delay between two processes. If, for example, one process is produced by another one by a simple lag, then the cross correlation function would have a peak at $\Delta t=t_{lag}$. This concept is important for AGN reverberation mapping, which we will discuss below.

Note that the cross correlation has other uses, not in the time domain.  For example, say you want a sample of spectra to be precisely lined up in wavelength.  Take the cross correlation with the mean spectrum, model the peak, and shift according to the shift of the peak.  This is accurate to a fraction of a bin, in contrast to, e.g., lining up spectra by eye, since all parts of the spectrum contribute to the offset, not just the major features.

The autocorrelation function is the same as the cross correlation function, with $f=g$. It yields information about the correlation of various time scales in the process. For example, in white noise, adjacent points are uncorrelated, and the autocorrelation function has a peak at $\Delta t$ equal zero, and is zero otherwise. Correlated processes, such as $1/f$ noise, have significant autocorrelation for larger $\Delta t$.

The idealized form of the _structure function_ is defined as

$$SF(\Delta t) = SF_{\infty} [1 - ACF(\Delta t)]^{1/2}, $$

where $SF_{\infty}$ is the standard deviation over an infinitely long time (or at least longer than a characteristic time scale). 

How does this work in the face of $1/f$ noise, where the power diverges on long and short timescales? Physically, a power of a process can't diverge, so a $1/f$ process must break to a flat process below some frequency, and cut off beyond some high frequency.  These break frequencies may reflect a natural time scales in the astronomical object.

The structure function and power spectral density are related; we noted above that Parseval's theorem states that the power measures either way must be the same. The Wiener-Khinchin theorem points out that they are Fourier pairs. So, if the structure function is described by $ SF \propto \Delta t^\alpha$, then the $PSD \propto 1/f^{1+2\alpha}$.

### Computing the Autocorrelation and Structure Functions

For evenly-sampled discrete data, the autocorrelation function is written:

$$ACF(j)=\frac{\sum_{i=1}^{N-j}[(y_i-\bar{y})(y_{i+j}-\bar{y})]}{\sum_{i=1}^{N}(y_i-\bar{y})^2},$$

where $i$ indexes the data point, and $j$ indexes the lag.  It can be visualized as shifting the light curve with respect to itself, multiplying the two light curves, and summing the result.

When data are unevenly sampled, the ACF above cannot be used. There are several ways proposed to deal with unevenly sampled data. A common one, called the "discrete correlation function" (DCF) was proposed by Edelson & Krolik (1988). The way it works is that the following value, the unbinned discrete correlations, is computed:

$$UDCF_{ij}=\frac{(a_i-\bar{a})(b_j-\bar{b})}{\sqrt{(\sigma_a^2-e_a^2)(\sigma_b^2-e_b^2)}},
$$
for all pairs in the two measured data trains $a_i$ and $b_i$.  (Implicit here is that these two data sets are sampled at the same time; reverberation data would be an example.). Each UDCF value is associated with a lag $\Delta t_{ij}=t_j-t_i$. The parameter $e$ is the measurement error.

The DCF, then, is a simple binning over time of the values of the UDCF in bins of width $\Delta \tau$. That is, the average is computed for the $M$ pairs that lie in this range: $\tau - \Delta \tau/2 \leq \delta t_{ij} \le \tau+\delta \tau/2$. Then

$$DCF(\tau)=\frac{1}{M} UDCF_{ij}.$$

The bin size has to be chosen for this method. It is a tradeoff between requiring good statistics (i.e., large M) in each bin (tending to increase the bin size), and requiring high time resolution (tending to decrease the bin size). Note that the zero-lag bin, here $i=j$, is excluded.

Edelson & Krolik provide an uncertainty on the DCF:

$$\sigma_{DCF}(\tau) = \frac{1}{M-1}\left (\sum \left [UDCF_{ij} -
DCF(\tau)\right ]^2\right )^{1/2},$$

but this is only strictly appropriate if there is no significant correlation over widths greater than the bin size. For power law power spectra (i.e., pink or red noise), that won't be the case. Simulations are always advised to estimate uncertainties.

The astroML package has an implementation of this algorithm that can be used for the autocorrelation, but, mystifyingly, it apparently does not have the algorithm for the cross correlation (i.e., what you would use to compute lags, which was actually what Edelson & Krolik were trying to do).

Scargle has a different, rather roundabout procedure to compute the autocorrelation and cross correlation. Ivezic outlines it as follows:

1. Compute the generalized Lomb-Scargle periodogram for the unevenly-sampled data.
2. Compute the sampling window function using the generalized Lomb-Scargle periodogram.
3. Compute the inverse fourier transform for both ($\rho(t)$ and $\rho^W(t)$).
4. Finally, compute the autocorrelation at lag $t$ equals $ACF(t)=\rho(t)/\rho^W(t)$.

Ivezic Figure 10.30, reproduced below, illustrates the use of these algorithms.

![Ivezic 10.30](http://www.astroml.org/_images/fig_autocorrelation_1.png)

    Ivezic Figure 10.30. An example of the autocorrelation function for a stocastic process. Top: the simulated light curve using a damped random walk model. The bottom panel shows autocorrelation computed using various algorithms.

## Example

Let's see how the autocorrelation function works for the time series discussed above.

In [ ]:
from astroML.time_series import ACF_scargle, ACF_EK


In [ ]:
N = 1024
dt = 0.01
factor = 100

t = dt * np.arange(N)
beta=1.0

x=factor*generate_power_law(N, dt, beta)
x2=factor*generate_power_law(N, dt, beta)

plt.subplot(1,2,1)

plt.plot(t,x)
plt.ylim(-1.5,1.5)
plt.subplot(1,2,2)

plt.plot(t,x2)
plt.ylim(-1.5,1.5)



In [ ]:
## Add some constant in order to generate some errors

xnew=20.0*x+100.0
x2new=20.0*x2+100.0
xerr=sqrt(xnew)
x2err=sqrt(x2new)

## Try normalizing

print xnew.std()

xnewnorm=(xnew-xnew.mean())/xnew.std(0)
xerrnorm=xerr/xnew.std(0)
plt.subplot(1,2,1)

#plt.plot(t,xnew/xerr)
#plt.errorbar(t,xnew,yerr=xerr)
plt.plot(t,xnewnorm)
plt.plot(t,xerrnorm)
#plt.ylim(0,140)
plt.subplot(1,2,2)

plt.plot(t,x2new)
plot,plot(t,x2err)
#plt.ylim(-1.5,1.5)




In [ ]:
C_EK, C_EK_err, M = ACF_EK(t,xnewnorm,xerrnorm/2.0,N)

In [ ]:
print M.shape
print C_EK.shape
print C_EK_err.shape

#plt.plot(M[1:],C_EK)
#plt.plot(M[1:],C_EK_err)

plt.errorbar(M[1:],C_EK,yerr=C_EK_err)
print C_EK

### Simulated Light Curves

It is advisable to create simulated light curves to significance-test models. Periodic light curves are straightforward - one starts with a periodic function and adds noise (e.g., replacing the point by drawing a random number from a poisson distribution if it is photon counting data, or subtracting a random number from a gaussian distribution with width typical of the homoscedastic undertainty.) But how is it done when the underlying power spectrum is stocastic?

The astroML package includes a routine to produce light curves from power-law power spectra based on the method of Timmer & Konig 1995.   This method is outlined as follows.

1. Choose a power law spectrum $S(\omega) \sim (1/\omega)^\beta$.
2. For each Fourier frequency, draw two gaussian distributed random numbers, multiply them by $(1/\omega)^{\beta/2}$, and replace the power spectrum amplitude by this randomized amplitude. Do it for both the real and imaginary parts. Fill the negative frequencies with appropriate symmetry.
3. Obtain the time series by taking the inverse Fourier transform.


Measurement noise may also be added by hand.

Typically, very long light curves are made that are split into a number of lightcurves the same length as the observed light curve. Examples of use of simulated light curves can be found in [Uttley et al. 2002](http://adsabs.harvard.edu/abs/2002MNRAS.332..231U).

Figure 10.29 from Ivezic illustrates some simulated time series and their power spectra.

![Ivezic Figure 10.29](http://www.astroml.org/_images/fig_powerlaw_1.png)
 
    Ivezic Figure 10.29. Example time series (top) and their power spectra (bottom). The left side shows the case for $1/f$ noise, the right shows random walk ($f1/f^2)$).

There are problems with this algorithm, though. Because it randomizes using a Gaussian distribution, the simulated light curves end up resembling the observed only in mean and variance. Higher order moments, such as skew, are not modelled. However, as has been pointed out, sometimes the observed light curves look bursty, i.e., with high skew. A newer algorithm has been proposed by [Emmanoulopoulos et al. 2013](http://adsabs.harvard.edu/abs/2013MNRAS.433..907E) which uses replacements to generate light curves that not only have the same power spectrum but also the same flux distribution (i.e., skew).

A typical simulation experiment would proceed as follows. Say, for example, you wanted to constrain the slope of the power spectrum of a light curve (or more interestingly, the cutoff frequency, which might have something to do with the process producing the variability). You would take the power spectrum of your object, and compare with the power spectra of a large number of simulated light curves via some figure of merit. This can be quite resource (time, computing) intensive, so it occurred to me that a possibly better way to do this would be with MCMC, i.e., explore parameter space. But someone has already done it:  [Marshall 2015](http://adsabs.harvard.edu/abs/2015ApJ...810...52M).

## Reverberation Mapping

### Background - Photoionization and the Broad Line Region

Photoionization line emission in astronomical objects is a pretty straightforward concept. 
 - Consider that you have an astronomical object that emits light with $E=h/\nu$ greater than 1 Rydberg.  
 - Consider that this astronomical object is located in the vicinity of some basically neutral gas. The hydrogen ionizing photons with $E > 1$ Rydberg will ionize hydrogen. The resulting free electrons, in essence, heat the gas. 
 - Other elements will be photo ionized according to their ionization potential. The ions will recombine, with a rate depending on the density of the emitting gas, emitting photons. 
 - They may also be collisionally excited by the free electrons, and then decay to the ground state by emission of a photon. These photons exit the excited gas, cooling the gas. Due to the quantization of energy levels, the photons emerge at discrete energies. They are the emission lines.


The example shown below is an HST image of the Orion Nebula. The source of ionizing photons is one of the Trapizium stars. The light shows in such emission lines as Hβ, the n=4 to n=2 transition in hydrogen, where n is the principal quantum number.

![HST orion nebula](https://upload.wikimedia.org/wikipedia/commons/f/f3/Orion_Nebula_-_Hubble_2006_mosaic_18000.jpg)

When we talk about reverberation mapping, we are usually talking about active galactic nuclei. 
 - Active galactic nuclei produce copious emission with $E>$ 1 Rydberg because they have hot, extreme-UV-emitting accretion disks. 
 - Within the active nucleus, unresolvable to telescopes, exists clouds of cool gas (the broad line region) that orbit in the potential of the supermassive black hole.  They orbit at a good clip and so the emission line are broad due to the  Doppler effect. 
 - The inner edge of the BLR is likely defined by where the clouds can no longer cool sufficiently by emitting lines, or where the accretion disk breaks up due to self gravity (and there are some other ideas). 
 - The outer edge is defined by where the radiation intensity has dropped enough that dust can exist. Below is a schematic diagram of an active nucleus (dramatically not to scale - basically the diagram is logarithmic).
 
![diagram of AGN](http://heasarc.gsfc.nasa.gov/docs/objects/agn/agn_model.gif)

And here is a typical spectrum of a quasar:

![quasar spectrum](https://ned.ipac.caltech.edu/level5/Glossary/Figures/figure2_2.gif)

Note much of the spectrum has wavelengths shorter than cannot be observed on the surface of the earth. They can be observed by _HST_, though, or they can be observed on the ground in higher redshift objects.

By itself, the broad line region isn't terribly interesting. What is interesting is what the BLR can tell you about the active nucleus.   

**Reverberation mapping** was proposed by [Blandford and McKee 1982](http://adsabs.harvard.edu/abs/1982ApJ...255..419B).  The idea is very simple.

- Quasars are observed to vary in their continuum emission, both in their optical and UV, and in their X-ray band. The origin of the variability is not known, but it may be caused by cascades of changing conditions due to fluctuations in accretion rate (deliberately vague). We therefore assume that they vary in the unobservable extreme UV, making them a varying source of photoionizing photons.
- Quasar emission lines are also observed to vary, and when monitored, it can be seen that they vary in concert with the continuum emission, but with a delay that is caused by the light travel time from the continuum source to the line emitting gas. An example seen below.

![lightcurve](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure24.jpg)

Blandford and McKee postulated that if the variability obeyed certain requirements (discussed below) and intensive and frequent monitoring were used, then the location and velocity of the broad-line emitting clouds could be extracted from the processed data. We will first discuss how it is done, and then discuss what you can learn from it.

This discussion follows [Brad Peterson's book on AGN](https://www.amazon.com/Introduction-Active-Galactic-Nuclei/dp/0521479118/ref=mt_paperback?_encoding=UTF8&me=&dpID=41-ImRf3xTL&preST=_SY344_BO1,204,203,200_QL70_&dpSrc=detail), [Julian Krolik's book on AGN](https://www.amazon.com/Active-Galactic-Nuclei-Princeton-Astrophysics/dp/0691011516), [this webpage](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Peter_contents.html), and [this webpage](https://ned.ipac.caltech.edu/level5/March10/Peterson/Peterson_contents.html).

Consider the following thought experiment. The quasar continuum varies as a delta function. The broad line region is located in a thin sphere some distance $r$ from the continuum emitting source. After a time $t=r/c$ (where $c$ is the speed of light), the delta function continuum pulse will reach the broad line region and excite the gas, causing it to emit emission lines. The continuum and line emission will travel toward us. We will observe the continuum as a delta function, but the line emission pulse (except for the emission from the gas right on our line of sight) will be delayed, with the lag depending where on the sphere the gas is.

A first assumption is that the gas will respond on shorter time scales than the other timescales (e.g., the continuum variability time scale) in the system. This requirement is easily met; the hydrogen recombination time scale depends on the gas density, and for typical quasar BLR densities, the timescale is on the order of minutes, while the lags are on orders of days/weeks.

The response of the emission line gas can be analyzed by looking at the isodelay surface, which is, not surprisingly, a parabola.

![parabolic delay](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure7.jpg)

Taken from Peterson 2001. The upper figure shows a cross section of a spherical BLR shell, a radius $r$ from the central engine. The parabolic isodelay surface is shown.

![parabolic](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure6.jpg)

Taken from Peterson 2001. The isodelay surfaces showing that the region nearer the observer appears to respond first, with the largest delay coming from the material directly opposite the observer, with a total lag of $2r/c$.

The lag is given by

$$\tau=(1+\cos \Theta) r/c.$$

The isodelay surface on a thin spherical shell is a ring with radius $r \sin \Theta$ with surface area $2 \pi r^2 \sin \Theta d\Theta$.  

Consider the "emission line response" to be the number of additional photons added to the line, and denote it as $\zeta$ (zeta). 

Then the emission line response seen by the observer is the product of the area and $\zeta$, i.e.,

$$\Psi(\theta) d\Theta = 2\pi \zeta r^2 \sin \Theta d\Theta.$$

Computing $d\tau/d\Theta$ yields:

$$d\tau = -(r/c) \sin \Theta d\Theta$$

and the line response can be written as a function of time delay by

$$\Psi(\tau) d\tau = \Psi(\Theta) |\frac{d\Theta}{d\tau}| d\tau = 2\pi
\zeta r c d\tau.$$

$\Psi(\tau)$ is the transfer function. This equation shows that for a thin sphere, the transfer function is simple; the line response varies from no lag (i.e., along the line of sight) to $\tau=2r/c$ from the far side of the central engine.

The equations become more complicated for more complex geometries. In addition, it is useful to also use the velocities of the BLR to obtain additional information (below).

Consider first clouds in a circular orbit at an inclination of 90 degrees from the viewer, as shown in the figure below:

![delay](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure8.jpg)

The line of sight velocities will be $V_Z=\pm V_{orb} \sin \Theta$, where $V_{orb}=(GM/r)^{1/2}$. For an arbitrary $i$, the circular orbit will project to an ellipse in the line-of-sight velocity/time-delay $(V_Z,\tau)$ plane with semiaxes $V_{orb}$ and $r/c$.

If you consider random-direction orbits with fixed speed on a thin spherical shell, you obtain the figure below (using Monte-carlo computations).

![more delays](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure9.jpg)

    Transfer function for a thin spherical shell. The transfer function marginalized over velocity (on the right) shows the time delay for the entire line, while the lower panel shows the response function integrated over time delay, or the profile of the variable part of the line.

This can be generalized further. What happens if the lines are not emitted isotropically, i.e., the emission is only occurring on the side of the cloud facing the target? Then, the emissivity of a cloud can be parameterized by

$$\epsilon(\Theta) = \epsilon_0 (1+A \cos\Theta),$$

where $\epsilon_0$ is a constant and $A=0$ for isotropic emission, and $A=1$ for completely anisotropic emission. The principal effect of this will be to increase the lag, because only the ones on the far side of the continuum source will be seen. The figure below shows the transfer function for this case:

![yet more](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure10.jpg)

Next consider the case where the BLR may be spherical, but the quasar continuum emission is not. It may be blocked along some directions, not reaching the clouds. For example, the BLR may be illuminated by biconical beams.

![biconical](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure11.jpg)

A further generalization comes because the BLR is extended in radius, i.e., a thick shell. A thick shell has $r_{in}$ and $r_{out}$, and also a distance-dependent responsivity $\epsilon(r) \propto \epsilon_0 r^\beta$. $\beta$ parameterizes several different physical effects:
- the dilution of the ionizing flux with radius that influences the flux but also the ionization state
- a radius-dependent covering fraction (noting that the BLR has to be confined to small clouds)
- potentially, radially-dependent density.

The situation for two rings is shown below:

![two ring](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure13.jpg)

The following two figures show the transfer fuction for the thick shell for two differing radial-response indices $\beta$.

![figure 1](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure14.jpg)

![figure 2](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure15.jpg)

These complications can be combined and further complicated. For example, the clouds may have an outflow as well as circular motion, the thick disk can be illuminated aniostropically, etc, etc.

The goal of reverberation mapping is to determine the transfer function from the emission line response to continuum variations, and from that infer the geometry of the broad-line region.

## The Reverberation Mapping Experiment

Having established that the continuum and the lines vary, and establishing that in principle different geometries result in different transfer functions (and we will discuss recovering the transfer function in a minute), we now list the requirements for a reverberation mapping experiment, and the assumptions.

Assumptions:

- We only map the gas that emits. There may be gas that does not emit effectively in that particular line, or does not vary in response to the continuum. Gas that is optically thin to the ionizing continuum tends to not vary because an increase in the ionizing flux is complensated by a decrease in the fraction of ions in the relevant ionization state. Additional issues associated with this are discussed by [Goad & Korista 2014](http://adsabs.harvard.edu/abs/2014MNRAS.444...43G).
- It is assumed that the ionizing continuum, which we cannot observe, follows the observed continuum variability, which may be in the optical for ground-based experiments, i.e., far from the ionizing continuum. In particular, it may be that the shape of the spectral energy distribution changes with flux, and our assumptions will likely not aacount for that. In addition, the assumptions will not account for the possibility that the continuum that the emission lines see is not the continuum that we see, and the whole broad-line region will see the same continuum emission (continuum emission isotropy assumption).
- The monitoring frequency must be much less than the reverberation / variability time scales, and the monitoring period must be long enough that significant variability events occur. The rule of thumb is that the monitoring period must be $\sim 3$ times longer than the lag to be measured. This ends up being a limit on the lumionsity of the monitored objects - low luminosity Seyfert galaxies (versus quasars) vary on weeks/months time scales required.
- High signal-to-noise ratio spectra are required. This pretty much rules out small aperture telescopes ($\sim 1$ m).

Reverberation mapping experiments tend to be quite resource intensive. [A recent one](http://adsabs.harvard.edu/abs/2015ApJ...806..129E) on the Seyfert galaxy NGC 5548 spanned 125 days, including approximately daily HST observations. Such space-based coverage is difficult and so only a handful of objects have been monitored in the UV.

Ground-based campaigns are easier, but because only low-luminosity objects can be targeted, they are limited to emission lines in the rest-optical wavelengths (such as H$\beta$, and HeII$\lambda$4685). For example, [another recent monitoring campaign](http://adsabs.harvard.edu/abs/2015ApJS..217...26B) used basically all the time on the 3m Shane Telescope at Lick Observatory for 2.5 months; 15 Seyferts were observed.

### Recovering the Transfer Function

Now that you have run the reverberation experiment, what do you do with all that data? Ideally, you want to recover the transfer function, since we have shown that it maps to the geometry of the broad-line region.

Reverberation mapping is, essentially, a convolution, suggesting that the convolution theorem could be used to obtain the transfer function. That is, the Fourier transform of the continuum is:

$$C^\star(f)=\int_{-\infty}^{\infty} C(t) e^{-i 2\pi f t}dt,$$

and the Fourier transform $L^\star(f)$ of the line can be similarly obtained. Then, the transfer function is, by the convolution theorem:

$$\Psi^\star(f)=L^\star(f)/C^\star(f),$$

and the transfer function can be obtained, in principle from the inverse Fourier transform:

$$\Psi(\tau)=\int_{-\infty}^{\infty} \Psi^\star(f) e^{-i 2\pi f t}
dt.$$

In practice, the data are often too noisy and too sparsely sampled to do this, and people simply use cross-correlation to get the _first moment_ of the transfer function (i.e., lag).

If the data is very good, then advanced methods can be used. These include the Maxmimum Entropy Method (e.g., [Horne et al. 1994](http://adsabs.harvard.edu/abs/1994ASPC...69...23H), [Horne et al. 2003](http://adsabs.harvard.edu/abs/2003MNRAS.339..367H)). Other methods include direct fits to light curves ([Ying et al. 2011](http://adsabs.harvard.edu/abs/2011ApJ...735...80Z)) or regularized linear inversion ([Skielboe et al. 2015](http://adsabs.harvard.edu/abs/2015MNRAS.454..144S).



### Cross Correlation Analysis

Cross correlation is like autocorrelation except that two different light curves are used. We already wrote the equation for cross correlation above:

$$CF(\Delta t) = \frac{\text{lim}_{T\rightarrow \infty} \frac{1}{T}
\int_{(T)} f(t) g(t+\Delta t) dt}{\sigma_f \sigma_g},$$

where $\sigma_f$ and $\sigma_g$ are the standard deviations of $f(t)$ and $g(t)$.

For reverberation mapping, we correlate the line-flux light curve with the continuum flux lightcurve. The figure above again applies:

![x-corr](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure24.jpg)

and the cross correlation between the line and continuum yields:

![x-corr 2](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure25.jpg)

    From Peterson 2001, and constructed from the light curve above. This shows that the emission line emitting region lies 15.6 light days from the central engine.

Let us examine specifically how the cross correlation is related to  the transfer function.

$$CCF(\tau) = \int_{-\infty}^{\infty} L(t)C(t-\tau) dt.$$

Recalling that $L(t) = \int_{-\infty}^{\infty} \Psi(\tau)
C(t-\tau)d\tau$, we obtain

$$CCF(\tau) = \int_{-\infty}^{\infty}
C(t-\tau)\int_{-\infty}^{\infty} \Psi(\tau^\prime)C(t-\tau^\prime)
d\tau^\prime dt.$$
$$ = \int_{-\infty}^{\infty}\Psi(\tau^\prime)\int_{-\infty}^{\infty}
C(t-\tau^\prime) C(t-\tau) dt \, d\tau^\prime.$$

The inner integral is the cross correlation of the continuum light curve with itself, i.e., the autocorrelation function $ACF(\tau)$. So the equation becomes:

$$CCF(\tau)=\int_{-\infty}^{\infty} \Psi(\tau^\prime)
ACF(\tau-\tau^\prime) d\tau^\prime.$$

If the line light curve is the continuum light curve simply offset by a lag, then you will be interested in is the position of the lag.  This can be considered to be the centroid of the CCF:

$$\tau_{cent} = \frac{\int \tau CCF(\tau)d\tau}{\int CCF(\tau)d\tau},
$$
which is related to the transfer function as:

$$\tau_{cent} = \frac{\int \tau \Psi(\tau)d\tau}{\int
\Psi(\tau)d\tau}.$$

Thus, the cross correlation can be seen to be the first moment of the transfer function. Note that this doesn't always give the greatest answer because in order to work in an absolute sense, the light curves have to be infinite. More typically, they sample only one variability event.

There are lots of computational issues that I won't go into. These include uneven sampling and how to deal with that (interpolation?) Another detail is how to measure the uncertainty in the lag. But once you are confident that you have your uncertainties under control, you can find some quite interesting results about AGN.

For example, we find that the broad-line region is stratified, with higher ionization lines being located closer to the central engine than lower ionization lines, as shown in the figure below:

![stratified](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure31.jpg)

    Taken from Peterson 2001. Lightcurves from NGC 5548. It can be seen that the high ionization lines (e.g., HeII) are emitted closer to the central engine that low ionization lines (e.g., H$\beta$).
    

Another important result is that the broad-line region size is correlated with the AGN continuum luminosity. An early result is shown below, and an updated result is shown in Bentz et al. 2013. The radius depends on the continuum luminosity to the $0.5$ power, as you would expect if the BLR is scale invariant.


![scale invariant](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure32.jpg)

    The broad line region radius as measured by H$\beta$ lag, as a function of average optical continuum luminosity. This is an early example; more recent data shows that the radius depends on the luminosity with a power law index of 0.5.

Another result is that an AGN black hole mass can be determined from the reverberation mapping results via the virial relationship:

$$M=\frac{f r \sigma^2}{G}, $$

where $f$ is a dimensionless factor of the order of unity that depends on the geometry and kinematics of the BLR, $\sigma$ is the emission-line velocity dispersion (i.e, the width of the H$\beta$ line, typically), and $r$ is the distance determined from the lag.  Basically, this is saying that, overall, the broad line region clouds have Keplerian orbits around the black hole, with the width of the line depending on the distance. The evidence for this is below:

![sketchy](https://ned.ipac.caltech.edu/level5/Sept01/Peterson2/Figures/figure33.jpg)


The really useful thing about this result is that now you don't need a whole reverberation mapping experiment to estimate the black hole mass - you just need to measure luminosity (to estimate the radius) the width of the line, and from those two numbers you can obtain a "single epoch black hole mass" estimate. This result stands on much shakier ground, basically because only a few objects have been monitored in the UV, a requirement to produce the figure above.



## Doppler Tomography

Some objects exhibit variable lines and continuum, but their size scale and brightness are such that their data are amenable to advanced data analysis techniques. An example is the class of binary stars known as Cataclysmic Variables (CVs), which have an evolved star and a white dwarf star in a binary orbit. The evolved star loses mass through Roche Lobe overflow, and that matter falls on to the white dwarf through an accretion disk. The accretion disk produces emission lines, and those lines are variable. By mapping the variability of the emission lines, the shape and properties of the disk can be inferred. A schematic diagram of a CV is below:

![CV](http://chandra.harvard.edu/photo/2001/v1494aql/aquilae_illustration_bl.jpg)

[Here](https://www.youtube.com/watch?v=774B8-9B4Ow) is an animation of one.

And a useful introductory but thorough webpage is found [here](http://cronodon.com/SpaceTech/CVAccretionDisc.html).

The idea behind Doppler tomography is that the CV accretion disk is slowly varying, but because the object is in a binary orbit, what we see is rapidly changing due to doppler shifts. So we get different views of the system, although keep in mind that these objects are unresolvable, so the view is in velocity space.

Consider a point-like source of emission in a binary. Assuming this has a motion parallel to the orbital plane, line emission will trace out a sinusoid around the mean velocity of the system (this is simply the doppler effect for an orbiting object). This observed sinusoid will be associated with a particular velocity vector in the binary, with its own phase and amplitude. The whole disk will be described by a large number of these sinusoids that will be overlapping and
blended.

In more detail, a given point in the binary is defined by its spatial position, but also by its velocity $(v_x, v_y)$. (These velocities should reference the inertial frame, rather than the rotating frame, of course). The velocity values are measured as at a particular point in orbital phase. Defining $\phi=0$ to be when the donor star is closest to use yields

$$v_R=\gamma - v_x \cos 2 \pi \phi + v_y \sin 2 \pi \phi,$$

where $\gamma$ is the systemic velocity of the star, and $v_R$ references our line of sight radial velocity.

From this perspective, one can imagine a velocity-space image of the system $I(v_x,v_y)$.

Then, the line flux observed between the radial velocities $v$ and $v+dv$ at orbital phase $\phi$ is obtained by integrating over all the regions of the image that have the correct radial velocity, producing the line flux density:

$$f(v,\phi)= \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} I(v_x,v_y)[g(v-v_R)
dV] dV_x dV_y,$$

where $g$ is a function of velocity representing the line profile from any point in the image. Ideally, $g$ is narrow, a delta function, and so the integrand will pick out only those regions where

$$v=v_R=\gamma-v_x \cos 2\pi \phi + v_y \sin 2\pi \phi.$$

Note, here $v_R$ here is referring to radial; that is, we only see a velocity contribution to the line from the radial component of the vector (e.g., Fig. 1 in Marsh 2005). So, this is a straight line in
$v_x$, $v_y$ coordinates, with different values of $v$ defining a family of straight parallel lines across an image. The bottom line is that the formation of the line profile at a particular phase can be thought as a projection of the image (in velocity space) along the direction defined by the orbital phase (e.g., Figure 3 in [Marsh 2001](http://adsabs.harvard.edu/abs/2001LNP...573....1M)).

Below is a figure that shows some of the details of this mapping from the position coordinates to the velocity coordinates (the tomogram):

![tomogram](http://cronodon.com/images/binary_star_tomogram_key.jpg)

    Note how positions 1 to 4 on the disk map to the tomogram. The velocity of point 1 is equal to the velocity of the disc at this point minus the velocity of the white dwarf which is swinging towards us. This velocity is entirely in the positive y-direction, and so the x- component of the velocity on the tomogram is zero. Point 2 has negative x-velocity due entirely to relative motion of the disc, but it also has a small negative y-velocity due to motion of the white dwarf. 
Taken from [here](http://cronodon.com/SpaceTech/CVAccretionDisc.html).

Notice how the tomogram turns the disk inside out (larger velocities are in the inside of the disk)

So, time series data (i.e., the line profile) taken as the white dwarf and evolved star orbit their common center of mass are mapped to the doppler tomogram. The data are often displayed as trailed spectra as (for example, Fig. 3 of [Marsh 2005](http://adsabs.harvard.edu/abs/2005Ap%26SS.296..403M)).

The difficult part comes from converting the line profiles to velocity maps, considering that they have noise, etc. That is, the job is the transformation of $f(v,\phi)$ (i.e., the velocity as a function of phase) to $I(v_x,v_y)$, the velocity image. There are several methods that are used. One is called "filtered back projection", and another is called the maximum entropy method.